In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [8]:
df = pd.read_csv(r'C:\Users\jaimv\Downloads\covidtest.csv')
bf = pd.read_csv(r'C:\Users\jaimv\Downloads\noncovid.csv')

In [9]:
df.rename({'date_died': 'outcome'}, axis=1, inplace=True)

In [10]:
cf = bf[["ID", "outcome", "age", "gendera", "hypertensive", "diabetes"]]

In [11]:
cf.rename({'ID': 'id', 'gendera': 'sex', 'hypertensive': 'hypertension'}, axis=1, inplace=True)

C:\Users\jaimv\anaconda3\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [12]:
transform={2:1,1:0}
df = df.replace({'sex':transform,'hypertension':transform, 'diabetes':transform})

In [13]:
cf = cf.replace({'sex':transform})

In [14]:
df['outcome'][df['outcome']!='9999-99-99']=1.0

C:\Users\jaimv\AppData\Local\Temp/ipykernel_49076/4033142567.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['outcome'][df['outcome']!='9999-99-99']=1.0


In [15]:
df['outcome'][df['outcome']!=1]=0.0

C:\Users\jaimv\AppData\Local\Temp/ipykernel_49076/3564422947.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['outcome'][df['outcome']!=1]=0.0


In [16]:
df['covid_status'] = 0
cf['covid_status'] = 1

In [17]:
df['outcome'] = df['outcome'].dropna().astype('uint8')

In [18]:
df

,id,outcome,age,sex,hypertension,diabetes,covid_status
0,16169f,0,27,1,1,1,0
1,1009bf,0,24,1,1,1,0
2,167386,0,54,0,1,1,0
3,0b5948,0,30,1,1,1,0
4,0d01b5,1,60,0,0,0,0
...,...,...,...,...,...,...,...
566597,01ff60,0,58,1,1,0,0
566598,047cd1,0,48,0,1,1,0
566599,1beb81,0,49,0,1,1,0
566600,16fb02,0,43,0,1,1,0


In [19]:
cf = cf.dropna()

In [20]:
cf['outcome'] = cf['outcome'].astype(int)

In [21]:
cf

,id,outcome,age,sex,hypertension,diabetes,covid_status
0,125047,0,72,0,0,1,1
1,139812,0,75,1,0,0,1
2,109787,0,83,1,0,0,1
3,130587,0,43,1,0,0,1
4,138290,0,75,1,1,0,1
...,...,...,...,...,...,...,...
1172,171130,0,62,0,1,1,1
1173,101659,0,78,0,0,1,1
1174,162069,0,85,1,1,1,1
1175,120967,0,79,1,0,1,1
